In [15]:
!pip install azureml-sdk==1.56.0

  Using cached azureml_sdk-1.56.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached azureml_core-1.56.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached azureml_dataset_runtime-1.56.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached azureml_train_core-1.56.0-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of azureml-sdk to determine which version is compatible with other requirements. This could take a while.


ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python >=3.5,<3.9; 1.10.0.post1 Requires-Python >=3.5,<3.9; 1.10.0.post2 Requires-Python >=3.5,<3.9; 1.11.0 Requires-Python >=3.5,<3.9; 1.11.0.post1 Requires-Python >=3.5,<3.9; 1.11.0.post2 Requires-Python >=3.5,<3.9; 1.12.0 Requires-Python >=3.5,<3.9; 1.12.0.post1 Requires-Python >=3.5,<3.9; 1.12.0.post2 Requires-Python >=3.5,<3.9; 1.13.0 Requires-Python >=3.5,<3.9; 1.13.0.post1 Requires-Python >=3.5,<3.9; 1.13.0.post2 Requires-Python >=3.5,<3.9; 1.14.0 Requires-Python >=3.5,<3.9; 1.14.0.post1 Requires-Python >=3.5,<3.9; 1.14.0.post2 Requires-Python >=3.5,<3.9; 1.15.0 Requires-Python >=3.5,<3.9; 1.15.0.post1 Requires-Python >=3.5,<3.9; 1.16.0 Requires-Python >=3.5,<3.9; 1.16.0.post1 Requires-Python >=3.5,<3.9; 1.16.0.post2 Requires-Python >=3.5,<3.9; 1.17.0 Requires-Python >=3.5,<3.9; 1.17.0.post1 Requires-Python >=3.5,<3.9; 1.18.0 Requires-Python >=3.5,<3.9; 1.18.0.post1 Requires-Python >=3

**Importing the dependencies**

In [1]:
import os
import json
import requests

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [3]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [4]:
print(resource_group)
print(workspace_name)
print(region)

ml-model-deploy-raza
deploy-ml-model-deploy-raza
centralindia


**Create a Resource Group from Azure Portal**

In [5]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.


Deploying KeyVault with name deploymlkeyvault9f93f2d8.
Deploying StorageAccount with name deploymlstoragebafd67d85.
Deploying AppInsights with name deploymlinsightsc0695029.
Deployed AppInsights with name deploymlinsightsc0695029. Took 55.58 seconds.
Deploying Workspace with name deploy-ml-model-deploy-raza.
Deployed Workspace with name deploy-ml-model-deploy-raza. Took 52.7 seconds.
Workspace deploy-ml-model-deploy-raza created


In [6]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [7]:
model_name='diabetes_prediction_model'

In [8]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [9]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [10]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [11]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [12]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\mohdr\AppData\Local\Temp\ipykernel_21204\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-16 21:02:53+05:30 Creating Container Registry if not exists..
2024-06-16 21:12:53+05:30 Registering the environment..
2024-06-16 21:12:57+05:30 Building image..
2024-06-16 21:38:02+05:30 Generating deployment configuration.
2024-06-16 21:38:02+05:30 Submitting deployment to compute..
2024-06-16 21:38:10+05:30 Checking the status of deployment diabetes-prediction-service..
2024-06-16 21:39:24+05:30 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [13]:
scoring_uri = service.scoring_uri

In [14]:
scoring_uri

'http://6b36e78e-17cd-44cf-a424-c250f9eb7c54.centralindia.azurecontainer.io/score'

**Cleaning up all the created resources**

In [15]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-06-16 21:47:45+05:30 Check and wait for operation (7fc0ff7a-7f8d-434a-8810-c73ca7fc83d0) to finish.
2024-06-16 21:47:49+05:30 Deleting service entity.
Succeeded


In [16]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


In [17]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources